In [1]:
import os
import gc
import glob
import numpy as np
import pandas as pd
import json
import torch
from torch.utils.data import DataLoader
from torchinfo import summary

from skimage.measure import label, regionprops

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from src.utils import *
from src.dataset import get_data_from_file, test_dataset

c:\Users\wittd\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,
c:\Users\wittd\anaconda3\Lib\site-packages\pytools\persistent_dict.py:63: RecommendedHashNotFoundWarning: Unable to import recommended hash 'siphash24.siphash13', falling back to 'hashlib.sha256'. Run 'python3 -m pip install siphash24' to install the recommended hash.
  warn("Unable to import recommended hash 'siphash24.siphash13', "


Cupy implementation is not available. Make sure you have the right version of Cupy and CUDA installed.
Optional dependecy Dask_image is not installed. Implementations using it will be ignored.


In [3]:
save_path = 'C:/7_felev/szakdoga'

runs = os.listdir(save_path)
# remove wandb folder
runs.remove('wandb')

dataframes = {}
aggregated_data = {}

for run in runs:
    run_path = os.path.join(save_path, run)
    results = glob.glob(os.path.join(run_path, 'test_result*.json'))
    
    # List to hold all dictionaries for the current run
    run_data = []
    
    for result in results:
        with open(result, 'r') as f:
            data = json.load(f)
            run_data.append(data)
    
    # Create a DataFrame for the current run
    run_df = pd.DataFrame(run_data)

    mean_df = run_df.mean().to_frame(name='mean')
    std_df = run_df.std().to_frame(name='std')
    
    # Combine mean and std into a single DataFrame
    aggregated_df = pd.concat([mean_df, std_df], axis=1)
    
    dataframes[run] = run_df
    aggregated_data[run] = aggregated_df

# Now dataframes contains the aggregated DataFrame for each run
for run in runs:
    print(run)
    print(aggregated_data[run])
    # print(dataframes[run].head())
    print()

UNet2_len8_doubleconv
                           mean        std
dice_score             0.355941   0.010389
detection_rate         0.704719   0.043520
avg_pred_cell_size  1744.422795  82.501735
std_pred_cell_size  1039.136244  95.393875
avg_mask_cell_size   738.853227   0.000000
std_mask_cell_size   619.908815   0.000000

UNet4_len1_singleconv
                           mean         std
dice_score             0.361941    0.008812
detection_rate         0.711793    0.028222
avg_pred_cell_size  1422.704910  199.209316
std_pred_cell_size   854.140881  130.015269
avg_mask_cell_size   687.055321    0.000000
std_mask_cell_size   603.114496    0.000000

UNet4_len8_doubleconv
                           mean         std
dice_score             0.361604    0.019674
detection_rate         0.734969    0.040802
avg_pred_cell_size  1517.317151  203.670426
std_pred_cell_size   878.032632  141.310499
avg_mask_cell_size   687.055321    0.000000
std_mask_cell_size   603.114496    0.000000

UNet4_len8_sin

In [3]:
# calculating the high resoulution test cell sizes

def calculate_test_cell_sizes():
    path = 'C:/7_felev/data'
    files = []
    for root, dirs, files_ in os.walk(path):
        for file in files_:
            file_path = os.path.join(root, file)
            if file.endswith('.npz'):
                files.append(file_path)

    assert len(files) == 163, "Dataset size should be 163"

    np.random.seed(42)
    files = np.random.permutation(files)

    test_files = files[:33]
    # print(test_files)
    # return test_files

    cell_sizes_label = []
    for file in test_files:
        bio, mask = get_data_from_file(file, 8)
        # print(file, bio.shape, mask.shape)

        # calculating the high resoulution test cell sizes
        true_mask_int = mask.astype(np.int32)
        labeled_label, num_cells_label = label(true_mask_int, return_num=True)

        cell_sizes_label.extend([region.area for region in regionprops(labeled_label)])

    scale = mask.shape[0] / 80
    cell_sizes_label = pixel_to_micrometer(cell_sizes_label, scale)

    avg_cell_size_label = np.mean(cell_sizes_label) if cell_sizes_label else 0
    std_cell_size_label = np.std(cell_sizes_label) if cell_sizes_label else 0

    return avg_cell_size_label, std_cell_size_label

mean, std = calculate_test_cell_sizes()
print(mean, std)

649.2377342347785 512.7229437467809


In [ ]:
def replot(save_path, run):
    config_path = os.path.join(save_path, run)
    config = load_config(config_path)
    testset = test_dataset(config)
    test_loader = DataLoader(testset, batch_size=config['batch_size'], shuffle=False)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    for i in range(5):
        run_path = os.path.join(config_path, f'run_{i+1}')
        best_model_path = os.path.join(run_path, 'best_model.pth')
        model = torch.jit.load(best_model_path)
        plot_path = os.path.join(run_path, 'plots')

        if config['tiling']:
            plot_results_tiles(test_loader, model, device, plot_path, 'Test image', config)
        else:
            plot_results_image(test_loader, model, device, plot_path, 'Test image', config)

        del model
        torch.cuda.empty_cache()
        gc.collect()
        

save_path = 'C:/7_felev/szakdoga'

# print('UNet8_len8_singleconv')
# replot(save_path, 'UNet8_len8_singleconv')
# print('UNet_len1_doubleconv')
# replot(save_path, 'UNet_len1_doubleconv')
# print('UNet_len8_doubleconv')
# replot(save_path, 'UNet_len8_doubleconv')
# print('UNet_len8_singleconv')
# replot(save_path, 'UNet_len8_singleconv')

UNet8_len8_singleconv
UNet_len1_doubleconv
UNet_len8_doubleconv
UNet_len8_singleconv
